# Microsoft Sentinel Graph API with Graphistry

This notebook demonstrates how to query Microsoft Sentinel Graph API and visualize threat intelligence data with Graphistry.

## Requirements

```bash
pip install graphistry[sentinel-graph]
```

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install graphistry[sentinel-graph]

## Setup

Import libraries and configure Graphistry.

In [ ]:
import graphistry
from azure.identity import InteractiveBrowserCredential

# Register with Graphistry
# IMPORTANT: Store credentials securely using environment variables
graphistry.register(
    api=3,
    protocol="https",
    server="hub.graphistry.com"
    # personal_key_id='YOUR_KEY_ID',
    # personal_key_secret='YOUR_KEY_SECRET'
)

print("✓ Graphistry configured")

## Configure Sentinel Graph API

Set up authentication to Microsoft Security Platform. This will open a browser window for interactive login.

In [ ]:
# Interactive browser authentication
credential = InteractiveBrowserCredential()

# Replace 'YourGraphInstance' with your actual graph instance name
g = graphistry.configure_sentinel_graph(
    graph_instance='YourGraphInstance',
    credential=credential
)

print("✓ Sentinel Graph configured")

## Example 1: Basic Graph Query

Query nodes and edges from your graph instance.

In [ ]:
# Basic query to get nodes and edges
query = """
MATCH (n)-[e]->(m)
RETURN *
LIMIT 50
"""

viz = g.sentinel_graph(query)
print(f"Query returned {len(viz._node)} nodes and {len(viz._edge)} edges")

viz.plot()

## Example 2: Inspect the Data

Examine the structure of nodes and edges returned.

In [ ]:
# Access node and edge DataFrames
print("=" * 80)
print("NODES")
print("=" * 80)
print(f"Shape: {viz._node.shape}")
print(f"Columns: {list(viz._node.columns)}")
print("\nSample nodes:")
display(viz._node.head(3))

print("\n" + "=" * 80)
print("EDGES")
print("=" * 80)
print(f"Shape: {viz._edge.shape}")
print(f"Columns: {list(viz._edge.columns)}")
print("\nSample edges:")
display(viz._edge.head(3))

## Example 3: Enhanced Visualization

Add visual encodings for better graph exploration.

In [ ]:
styled = (
    viz
    .encode_edge_color('edge', as_categorical=True)
    .encode_point_color('label', as_categorical=True)
    .encode_point_size('label', default_mapping=100)
)

styled.plot()

## Example 4: Query with Filters

Use WHERE clause to filter results.

In [ ]:
# Query with WHERE clause (adjust property name as needed for your graph)
filtered_query = """
MATCH (a)-[e]->(b)
WHERE a.id IS NOT NULL
RETURN *
LIMIT 30
"""

filtered_viz = g.sentinel_graph(filtered_query)
print(f"Found {len(filtered_viz._edge)} edges")

filtered_viz.plot()

## Example 5: Query Nodes Only

Retrieve specific nodes from the graph.

In [ ]:
# Query nodes only
nodes_query = """
MATCH (n)
RETURN n
LIMIT 20
"""

nodes_viz = g.sentinel_graph(nodes_query)
nodes_viz.plot()

## Example 6: Error Handling

Demonstrate robust error handling.

In [ ]:
try:
    # Invalid query syntax
    bad_query = "INVALID SYNTAX"
    result = g.sentinel_graph(bad_query)
except Exception as e:
    print(f"Query failed as expected: {type(e).__name__}")
    print(f"Error message: {e}")

## Alternative Authentication: Service Principal

For production environments, use service principal authentication with credentials stored securely.

In [ ]:
# Uncomment and configure for production use
#
# import os
# 
# g_prod = graphistry.configure_sentinel_graph(
#     graph_instance='YourGraphInstance',  # Replace with your graph instance name
#     tenant_id=os.environ.get('AZURE_TENANT_ID'),
#     client_id=os.environ.get('AZURE_CLIENT_ID'),
#     client_secret=os.environ.get('AZURE_CLIENT_SECRET')
# )
# 
# result = g_prod.sentinel_graph('MATCH (n) RETURN n LIMIT 10')
# result.plot()

## Cleanup

Clear cached authentication token.

In [ ]:
g.sentinel_graph_close()
print("✓ Sentinel Graph connection closed")